# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [14]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "phi3:latest"
openRouter_url = "https://openrouter.ai/api/v1"

In [ ]:
#set up environment

load_dotenv(override=True)
openRouter_api_key = os.getenv("OPEN_ROUTER_API_KEY")
if openRouter_api_key and openRouter_api_key.startswith("sk-") and len(openRouter_api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please check your environment variable!")

In [ ]:
question = """
Please explain what this code does and why:
def findCatalan(n):
  
    # Base case
    if n <= 1:
        return 1

    # catalan(n) is sum of catalan(i) * catalan(n-i-1)
    res = 0
    for i in range(n):
        res += findCatalan(i) * findCatalan(n - i - 1)

    return res


n = 6
res = findCatalan(n)
print(res)}
"""

In [ ]:
#Prompts for model

system_prompt = """
You are a helpful technical tutor who answers questions about provided code or technical programming questions, software engineering, 
data science and LLMs. And other related topics to computer science.

Note:
- If question didn't specify which language to be explained in, explain it in programming language it is written in.
- If question didn't specify what to explain, explain the whole code or the part that you think is most important
- Explain necessary concepts and terms. unless user ask deep explanation to whole question or on specific question
"""
user_prompt = "Please give a detailed explanation to the following question:\n\n" + question

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
#Generate solution with streaming method for displaying token per token.

openai = OpenAI(base_url=openRouter_url,api_key=openRouter_api_key)
stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)
response = ""

display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:

#generate a solution with ollama
try:
    import ollama
    print("Ollama package ready. Run 'ollama pull gemma3:270m' (or phi3) if you haven't already.")
except ImportError:
    ollama = None
    print("Install ollama package for Llama: pip install ollama")

if ollama is not None:
    response_llama = ollama.chat(model=MODEL_LLAMA, messages=messages)
    reply = response_llama["message"]["content"]
    display(Markdown(reply))
else:
    print("Skipping Llama: ollama package not installed.")